# Задание 1 (2 балла)

Напишите класс `MyDict`, который будет полностью повторять поведение обычного словаря, за исключением того, что при итерации мы должны получать и ключи, и значения.

**Модули использовать нельзя**

In [ ]:
class MyDict(dict):
    def __iter__(self):
        return iter(self.items())
    

In [ ]:
dct = MyDict({"a": 1, "b": 2, "c": 3, "d": 25})
for key, value in dct:
    print(key, value)   

a 1
b 2
c 3
d 25


In [ ]:
for key, value in dct.items():
    print(key, value)

a 1
b 2
c 3
d 25


In [ ]:
for key in dct.keys():
    print(key)

a
b
c
d


In [ ]:
dct["c"] + dct["d"]

28

# Задание 2 (2 балла)

Напишите функцию `iter_append`, которая "добавляет" новый элемент в конец итератора, возвращая итератор, который включает изначальные элементы и новый элемент. Итерироваться по итератору внутри функции нельзя, то есть вот такая штука не принимается
```python
def iter_append(iterator, item):
    lst = list(iterator) + [item]
    return iter(lst)
```

**Модули использовать нельзя**

In [ ]:
def iter_append(iterator, item):
    yield from iterator
    yield item

In [ ]:
my_iterator = iter([1, 2, 3])
new_iterator = iter_append(my_iterator, 4)

for element in new_iterator:
    print(element)

1
2
3
4


# Задание 3 (5 баллов)

Представим, что мы установили себе некотурую библиотеку, которая содержит в себе два класса `MyString` и `MySet`, которые являются наследниками `str` и `set`, но также несут и дополнительные методы.

Проблема заключается в том, что библиотеку писали не очень аккуратные люди, поэтому получилось так, что некоторые методы возвращают не тот тип данных, который мы ожидаем. Например, `MyString().reverse()` возвращает объект класса `str`, хотя логичнее было бы ожидать объект класса `MyString`.

Найдите и реализуйте удобный способ сделать так, чтобы подобные методы возвращали экземпляр текущего класса, а не родительского. При этом **код методов изменять нельзя**

**+3 дополнительных балла** за реализацию того, чтобы **унаследованные от `str` и `set` методы** также возвращали объект интересующего нас класса (то есть `MyString.replace(..., ...)` должен возвращать `MyString`). **Переопределять методы нельзя**

**Модули использовать нельзя**

In [ ]:
def self_type(cls_obj):
    # defining a changer
    def change_type(func):
        def inner_func(self, *args, **kwargs):
            # result of our method
            result = func(self, *args, **kwargs)
            if isinstance(result, (str, set)):
                class_obj = self.__class__
                result = class_obj(result)
            return result
        return inner_func
    
    # applying it when we're calling any method of this class
    for method_or_attr_name in dir(cls_obj):
        method_or_attr = getattr(cls_obj, method_or_attr_name)
        if callable(method_or_attr) and not method_or_attr_name.startswith('__'):
            setattr(cls_obj, method_or_attr_name, change_type(method_or_attr))
    
    return cls_obj


@self_type
class MyString(str):
    def reverse(self):
        return self[::-1]
    
    def make_uppercase(self):
        return "".join([chr(ord(char) - 32) if 97 <= ord(char) <= 122 else char for char in self])
    
    def make_lowercase(self):
        return "".join([chr(ord(char) + 32) if 65 <= ord(char) <= 90 else char for char in self])
    
    def capitalize_words(self):
        return " ".join([word.capitalize() for word in self.split()])
    
@self_type 
class MySet(set):
    def is_empty(self):
        return len(self) == 0
    
    def has_duplicates(self):
        return len(self) != len(set(self))
    
    def union_with(self, other):
        return self.union(other)
    
    def intersection_with(self, other):
        return self.intersection(other)
    
    def difference_with(self, other):
        return self.difference(other)

In [ ]:
string_example = MyString("Aa Bb Cc")
set_example_1 = MySet({1, 2, 3, 4})
set_example_2 = MySet({3, 4, 5, 6, 6})

In [ ]:
j = string_example.reverse()

In [ ]:
type(j)

__main__.MyString

In [ ]:
print(type(string_example.reverse()))
print(type(string_example.make_uppercase()))
print(type(string_example.make_lowercase()))
print(type(string_example.capitalize_words()))
print()
print(type(set_example_1.is_empty()))
print(type(set_example_2.has_duplicates()))
print(type(set_example_1.union_with(set_example_2)))
print(type(set_example_1.difference_with(set_example_2)))

<class '__main__.MyString'>
<class '__main__.MyString'>
<class '__main__.MyString'>
<class '__main__.MyString'>

<class 'bool'>
<class 'bool'>
<class '__main__.MySet'>
<class '__main__.MySet'>


# Задание 4 (5 баллов)

Напишите декоратор `switch_privacy`:
1. Делает все публичные **методы** класса приватными
2. Делает все приватные методы класса публичными
3. Dunder методы и защищённые методы остаются без изменений
4. Должен работать тестовый код ниже, в теле класса писать код нельзя

**Модули использовать нельзя**

In [ ]:
def switch_privacy(cls_obj):
    for method_or_attr_name in dir(cls_obj):
        method_or_attr = getattr(cls_obj, method_or_attr_name)
        if callable(method_or_attr):
            if method_or_attr_name.startswith(f'_{cls_obj.__name__}__'):  # private --> public
                # changing for a new name
                setattr(cls_obj, method_or_attr_name[method_or_attr_name.find('__')+2:],
                        method_or_attr)
                # deleting the old name
                delattr(cls_obj, method_or_attr_name)
            if method_or_attr_name[0] != '_':  # public --> private
                setattr(cls_obj, f'_{cls_obj.__name__}__{method_or_attr_name}',
                        method_or_attr)
                delattr(cls_obj, method_or_attr_name)
    
    return cls_obj

@switch_privacy
class ExampleClass:
    # Но не здесь
    def public_method(self):
        return 1
    
    def _protected_method(self):
        return 2
    
    def __private_method(self):
        return 3
    
    def __private_method2(self):
        return 3
    
    def __dunder_method__(self):
        pass

In [ ]:
test_object = ExampleClass()

test_object._ExampleClass__public_method()   # Публичный метод стал приватным

1

In [ ]:
test_object.private_method()   # Приватный метод стал публичным

3

In [ ]:
test_object._protected_method()   # Защищённый метод остался защищённым

2

In [ ]:
test_object.__dunder_method__()   # Дандер метод не изменился

In [ ]:
hasattr(test_object, "public_method"), hasattr(test_object, "private")   # Изначальные варианты изменённых методов не сохраняются

(False, False)

# Задание 5 (7 баллов)

Напишите [контекстный менеджер](https://docs.python.org/3/library/stdtypes.html#context-manager-types) `OpenFasta`

Контекстные менеджеры это специальные объекты, которые могут работать с конструкцией `with ... as ...:`. В них нет ничего сложного, для их реализации как обычно нужно только определить только пару dunder методов. Изучите этот вопрос самостоятельно

1. Объект должен работать как обычные файлы в питоне (наследоваться не надо, здесь лучше будет использовать **композицию**), но:
    + При итерации по объекту мы должны будем получать не строку из файла, а специальный объект `FastaRecord`. Он будет хранить в себе информацию о последовательности. Важно, **не строки, а именно последовательности**, в fasta файлах последовательность часто разбивают на много строк
    + Нужно написать методы `read_record` и `read_records`, которые по смыслу соответствуют `readline()` и `readlines()` в обычных файлах, но они должны выдавать не строки, а объект(ы) `FastaRecord`
2. Конструктор должен принимать один аргумент - **путь к файлу**
3. Класс должен эффективно распоряжаться памятью, с расчётом на работу с очень большими файлами
    
Объект `FastaRecord`. Это должен быть **датакласс** (см. про примеры декораторов в соответствующей лекции) с тремя полями:
+ `seq` - последовательность
+ `id_` - ID последовательности (это то, что в фаста файле в строке, которая начинается с `>` до первого пробела. Например, >**GTD326487.1** Species anonymous 24 chromosome) 
+ `description` - то, что осталось после ID (Например, >GTD326487.1 **Species anonymous 24 chromosome**)


Напишите демонстрацию работы кода с использованием всех написанных методов, обязательно добавьте файл с тестовыми данными в репозиторий (не обязательно большой)

**Можно использовать модули из стандартной библиотеки**

In [ ]:
import os
from dataclasses import dataclass


@dataclass
class FastaRecord:
    seq: str
    id_: str
    description: str

    def __repr__(self):
        return f'>{self.id_} {self.description} \n{self.seq} \n'


class OpenFasta:
    def __init__(self, file_path, mode):
        self.file_path = file_path
        self.mode = mode
        self.file = None
        self.next_header = None

    def __enter__(self):
        self.file = open(self.file_path, self.mode)
        return self

    def __exit__(self, exc_type, exc_value, exc_traceback):
        return self.file.close()

    def read_record(self):
        '''
        Since on each iteration we will 'cut out' the description of the next entry, we will store it in
        self.next_header - if we are not in the first entry, we'll infer description from this parameter.
        '''
        if self.next_header:
            first_seq = False
            id_name = self.next_header[1:self.next_header.find(' ')]
            descr = self.next_header[self.next_header.find(' ') + 1:]
        else:
            first_seq = True
        
        '''
        Main loop: abrupt iteration if we face >, else adding lines to seq attribute. If the seq remains empty, 
        we reached the end of the file.
        '''
        seq = ''
        for line in self.file:
            line = line.strip()
            # if there is a new entry
            if line.startswith('>'):
                if first_seq:
                    id_name = line[1:line.find(' ')]
                    descr = line[line.find(' ')+1:]
                    first_seq = False
                else:
                    self.next_header = line
                    break
            # if we have't reached the id line yet --> first_seq = True
            # or if we have --> first_seq = False
            else:
                if not first_seq:
                    # if line is not empty - adding it
                    if line:
                        seq = seq + line
                else:
                    continue

        if not seq:
            raise AttributeError('Empty sequence!')

        return FastaRecord(seq, id_name, descr)

    
    def read_records(self):
        try:
            while True:
                record = self.read_record()
                yield record
        except AttributeError as ar:
            pass

In [ ]:
with OpenFasta(os.path.join("data", "example.fasta"),'r') as fasta:
    # we're reading line by line and closing a file after usage -- space eficient performance
    for record in fasta.read_records():
        print(record)

>KF848938.1 Influenza A virus (A/USA/RVD1_H3/2011(H3N2)) segment 4 hemagglutinin (HA) gene, part 1 
CAAAAACTTCCTGGAAATGACAACAGCACGGCAACGCTGTGCCTTGGGCACCATGCAGTGCCAAACGGAACAATAGTGAAAACAATCACGAATGACCAAATTGAAGTTACTAATGCCACTGAGCTGGTTCAGAGTTCCTCAACAGGTGAAATATGCAACAGTCCTCATCAGATCCTTGATGGAGAAAACTGCACACTAATAGATGCTCTATTGGGAGACCCTCAGTGTGATGGCTTCCAAAACAAGAAATGGGACCTTTTTGTTGAACGAAGCAAAGCCC 

>KF848938.1 Influenza A virus (A/USA/RVD1_H3/2011(H3N2)) segment 4 hemagglutinin (HA) gene, part 2 
ACAGCAACTGTTACCCTTATGATGTGCCGGATTATGCCTCCCTTAGGTCACTAGTTGCCTCATCCGGCACACTGGAGTTTAACAATGAAAGCTTCAATTGGACTGGAGTCACTCAAAACGGAACAAGCTCTGCTTGCATAAGGAGATCTAATAATAGTTTCTTTAGTAGATTGAATTGGTTGACCCACTTAAACTTCAAATACCCAGCATTGAACGTGACTATGCCAAACAATGAACAATTTGACAAATTGTACATTTGGGGGGTTCACCACCCGGGTACGGACAAGGACCAAATCTTCCTGTATGCTCAAGCAGCAGGAAGAATCACAGTATCTACCAAAAGAAGCCAA 

>KF848938.1 Influenza A virus (A/USA/RVD1_H3/2011(H3N2)) segment 4 hemagglutinin (HA) gene, part 3 
CAAGCTGTAATTCCGAATATCGGATCTAGACCCAGAGTAAGGAATATCCCTAGCAGAGTAAGCA

In [ ]:
with OpenFasta(os.path.join("data", "example.fasta"), 'r') as fasta:
    print('The first record:')
    print(fasta.read_record())

The first record:
>KF848938.1 Influenza A virus (A/USA/RVD1_H3/2011(H3N2)) segment 4 hemagglutinin (HA) gene, part 1 
CAAAAACTTCCTGGAAATGACAACAGCACGGCAACGCTGTGCCTTGGGCACCATGCAGTGCCAAACGGAACAATAGTGAAAACAATCACGAATGACCAAATTGAAGTTACTAATGCCACTGAGCTGGTTCAGAGTTCCTCAACAGGTGAAATATGCAACAGTCCTCATCAGATCCTTGATGGAGAAAACTGCACACTAATAGATGCTCTATTGGGAGACCCTCAGTGTGATGGCTTCCAAAACAAGAAATGGGACCTTTTTGTTGAACGAAGCAAAGCCC 



# Задание 6 (7 баллов)

1. Напишите код, который позволит получать все возможные (неуникальные) генотипы при скрещивании двух организмов. Это может быть функция или класс, что вам кажется более удобным.

Например, все возможные исходы скрещивания "Aabb" и "Aabb" (неуникальные) это

```
AAbb
AAbb
AAbb
AAbb
Aabb
Aabb
Aabb
Aabb
Aabb
Aabb
Aabb
Aabb
aabb
aabb
aabb
aabb
```

2. Напишите функцию, которая вычисляет вероятность появления определённого генотипа (его ожидаемую долю в потомстве).
Например,

```python
get_offspting_genotype_probability(parent1="Aabb", parent2="Aabb", target_genotype="Аabb")   # 0.5

```

3. Напишите код, который выводит все уникальные генотипы при скрещивании `'АаБбввГгДдЕеЖжЗзИиЙйккЛлМмНн'` и `'АаббВвГгДДЕеЖжЗзИиЙйКкЛлМмНН'`, которые содержат в себе следующую комбинацию аллелей `'АаБбВвГгДдЕеЖжЗзИиЙйКкЛл'`
4. Напишите код, который расчитывает вероятность появления генотипа `'АаБбввГгДдЕеЖжЗзИиЙйккЛлМмНн'` при скрещивании `АаБбВвГгДдЕеЖжЗзИиЙйКкЛлМмНн` и `АаБбВвГгДдЕеЖжЗзИиЙйКкЛлМмНн`

Важные замечания:
1. Порядок следования аллелей в случае гетерозигот всегда должен быть следующим: сначала большая буква, затем маленькая (вариант `AaBb` допустим, но `aAbB` быть не должно)
2. Подзадачи 3 и 4 могут потребовать много вычислительного времени (до 15+ минут в зависимости от железа), поэтому убедитесь, что вы хорошо протестировали написанный вами код на малых данных перед выполнением этих задач. Если ваш код работает **дольше 20 мин**, то скорее всего ваше решение не оптимально, попытайтесь что-нибудь оптимизировать. Если оптимальное решение совсем не получается, то попробуйте из входных данных во всех заданиях убрать последний ген (это должно уменьшить время выполнения примерно в 4 раза), но **за такое решение будет снято 2 балла**
3. Несмотря на то, что подзадания 2, 3 и 4 возможно решить математически, не прибегая к непосредственному получению всех возможных генотипов, от вас требуется именно brute-force вариант алгоритма

**Можно использовать модули из стандартной библиотеки питона**, но **за выполнение задания без использования модулей придусмотрено +3 дополнительных балла**

In [2]:
          
def check_allelic_states_number(parent, ploidy=None):
    """
    Checks that all genes have the same number of allelic states (can be compared with the defined ploidy). 
    Returns ploidy and the checking result; example result: (True, 2)
    """
    parent_wo_states = parent.lower()
    allelic_states_numbers, checking_results = [], []
    for gene in set(parent_wo_states):
        allelic_states_numbers.append(parent_wo_states.count(gene))
    if ploidy is None: 
        ploidy = allelic_states_numbers[0]
    return all(some_gene_states_n == ploidy for some_gene_states_n in allelic_states_numbers), ploidy
        
        
def get_offspting_genotype_probability(parent1, parent2, target_genotype):
    '''
    Takes two parent genotypes and target genotype, then calculates the probability
    to get this genotype by crossing parents. 
    '''
    all_child_genotypes = all_genotypes(parent1, parent2)
    return all_child_genotypes.count(target_genotype)/len(all_child_genotypes)

    
def all_genotypes(parent1, parent2, containing=None):
    '''
    Takes genotypes of two parents and returns the list of all possible genotypes
    of offspring resulting from the cross.
    '''
    
    # checkig if the number of genes is equal for two parents
    genes1 = set(parent1.upper())
    genes2 = set(parent2.upper())
    if len(genes1.intersection(genes2)) != len(genes1):
        raise ValueError('Parents have different genes set!')
    
    # checking the ploidy
    for gene in genes1:
        if (parent1.upper().count(gene) != 2) or (parent2.upper().count(gene) != 2):
            raise ValueError('Please, provide diploid genotypes!')
            
    # dicts with alleles for each parent
    alleles_dicts = [dict(), dict()]
    for parent_id in [0, 1]:
        # alleles_dicts[parent_id] = dict()
        for gene in genes1:
            alleles_dicts[parent_id][gene] = [allele for allele in (parent1, parent2)[parent_id] if allele.upper()==gene]
    
    # dict with all combinations alleles for each gene 
    gene_alleles_dict = dict()
    for gene in genes1:
        gene_alleles_dict[gene] = []
        for allele1 in alleles_dicts[0][gene]:
            for allele2 in alleles_dicts[1][gene]: 
                allele_sm, allele_big = (allele1, allele2) if allele1 > allele2 else (allele2, allele1)
                gene_alleles_dict[gene].append(allele_big+allele_sm)
    
    # sorting in the alphabetical order
    gene_alleles_dict = dict(sorted(gene_alleles_dict.items(), key=lambda item: item[0]))

    if containing:
        # if some of required alleles are impossible, it's not feasible
        alleles = [containing[i:i+2] for i in range(0, len(containing), 2)]
        for allele in alleles:
            if allele not in gene_alleles_dict[allele[0].upper()]:
                print('It is impossible to get this combination from given genotypes')
                return 0
            
        # combining alleles
        offspring_genotypes = ['']
        for gene in gene_alleles_dict.keys():
            if gene in set(containing.upper()):
                offspring_genotypes = [
                    ''.join([prev, extension]) for prev in offspring_genotypes for extension in gene_alleles_dict[gene] if extension in alleles
                ]
            else:
                offspring_genotypes = [
                    ''.join([prev, extension]) for prev in offspring_genotypes for extension in gene_alleles_dict[gene]
                ]    

    else:
        offspring_genotypes = ['']
        for gene in gene_alleles_dict.keys():
            offspring_genotypes = [
                ''.join([prev, extension]) for prev in offspring_genotypes for extension in gene_alleles_dict[gene]
            ]
    return offspring_genotypes

In [2]:
# task 1
all_genotypes('Aabb', 'AaBb')

['AABb',
 'AAbb',
 'AABb',
 'AAbb',
 'AaBb',
 'Aabb',
 'AaBb',
 'Aabb',
 'AaBb',
 'Aabb',
 'AaBb',
 'Aabb',
 'aaBb',
 'aabb',
 'aaBb',
 'aabb']

In [3]:
# task 3
all_genotypes('Aabb', 'AaBb', containing='AA')

['AABb', 'AAbb', 'AABb', 'AAbb']

In [4]:
# task 2
get_offspting_genotype_probability("Aabb", "Aabb", target_genotype="Aabb") 

0.5

In [5]:
%%time
# task 3
crosses_filtered = all_genotypes('АаБбввГгДдЕеЖжЗзИиЙйккЛлМмНн', 'АаббВвГгДДЕеЖжЗзИиЙйКкЛлМмНН', 
                               containing='АаБбВвГгДдЕеЖжЗзИиЙйКкЛл')
len(crosses_filtered)

CPU times: user 27.1 ms, sys: 5.12 ms, total: 32.2 ms
Wall time: 41.1 ms


65536

Работало больше 20 минут, лучше не запускать:

In [1]:
# task 4
get_offspting_genotype_probability('АаБбВвГгДдЕеЖжЗзИиЙйКкЛлМмНн', 'АаБбВвГгДдЕеЖжЗзИиЙйКкЛлМмНн',
                                   target_genotype='АаБбввГгДдЕеЖжЗзИиЙйккЛлМмНн')

1.52587890625e-05
